In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe as dd
from datetime import datetime
import pytz
import seaborn as sns
from matplotlib.colors import LogNorm
import gc
#import awswrangler as wr
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

In [66]:
import os
os.chdir("C:\\Users\\paulo\\OneDrive\\Documentos\\Alaya")
os.getcwd()

'C:\\Users\\paulo\\OneDrive\\Documentos\\Alaya'

In [67]:
data_tot = pd.read_csv('MiningProcess_Flotation_Plant_Database1.csv', sep=';', decimal = '.')

In [68]:
# Checking data size and types
print(data_tot.dtypes)


date                             object
% Iron Feed                     float64
% Silica Feed                   float64
Starch Flow                     float64
Amina Flow                      float64
Ore Pulp Flow                   float64
Ore Pulp pH                     float64
Ore Pulp Density                float64
Flotation Column 01 Air Flow    float64
Flotation Column 02 Air Flow    float64
Flotation Column 03 Air Flow    float64
Flotation Column 04 Air Flow    float64
Flotation Column 05 Air Flow    float64
Flotation Column 06 Air Flow    float64
Flotation Column 07 Air Flow    float64
Flotation Column 01 Level       float64
Flotation Column 02 Level       float64
Flotation Column 03 Level       float64
Flotation Column 04 Level       float64
Flotation Column 05 Level       float64
Flotation Column 06 Level       float64
Flotation Column 07 Level       float64
% Iron Concentrate              float64
% Silica Concentrate            float64
dtype: object


In [69]:
data_tot.head(3)

,date,% Iron Feed,% Silica Feed,Starch Flow,Amina Flow,Ore Pulp Flow,Ore Pulp pH,Ore Pulp Density,Flotation Column 01 Air Flow,Flotation Column 02 Air Flow,...,Flotation Column 07 Air Flow,Flotation Column 01 Level,Flotation Column 02 Level,Flotation Column 03 Level,Flotation Column 04 Level,Flotation Column 05 Level,Flotation Column 06 Level,Flotation Column 07 Level,% Iron Concentrate,% Silica Concentrate
0,2017-03-10 01:00:00,55.2,16.98,3019.53,557.434,395.713,10.0664,1.74,249.214,253.235,...,250.884,457.396,432.962,424.954,443.558,502.255,446.370,523.344,66.91,1.31
1,2017-03-10 01:00:00,55.2,16.98,3024.41,563.965,397.383,10.0672,1.74,249.719,250.532,...,248.994,451.891,429.560,432.939,448.086,496.363,445.922,498.075,66.91,1.31
2,2017-03-10 01:00:00,55.2,16.98,3043.46,568.054,399.668,10.0680,1.74,249.741,247.874,...,248.071,451.240,468.927,434.610,449.688,484.411,447.826,458.567,66.91,1.31


In [70]:
y = data_tot.loc[:, '% Silica Concentrate'] 
y.value_counts()


2.080000    17100
1.190000    10080
3.110000     8460
1.340000     6840
1.680000     6300
            ...  
2.886676        1
4.925039        1
3.462767        1
5.468392        1
4.810884        1
Name: % Silica Concentrate, Length: 55569, dtype: int64

In [71]:
## Creating DataFrame for regression 

data_reg=data_tot.drop(['% Iron Feed','date','% Silica Concentrate'], axis=1)


In [72]:
#Data Partition

X_train, X_test, y_train, y_test = train_test_split(data_reg, y, train_size = 0.75, random_state = 123)

In [73]:
# Removing highly correlated data, in case is needed

# create correlation  matrix
corr_matrix = data_reg.corr().abs()

# select upper traingle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


# Find index of columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# drop the columns
#data_reg.drop(data_reg.columns[to_drop], axis=1)
data_reg=data_reg.drop(['Flotation Column 03 Air Flow'], axis=1)
print(to_drop)

['Flotation Column 03 Air Flow']


In [74]:
print(data_reg.info)

<bound method DataFrame.info of         % Silica Feed  Starch Flow  Amina Flow  Ore Pulp Flow  Ore Pulp pH  \
0               16.98      3019.53     557.434        395.713     10.06640   
1               16.98      3024.41     563.965        397.383     10.06720   
2               16.98      3043.46     568.054        399.668     10.06800   
3               16.98      3047.36     568.665        397.939     10.06890   
4               16.98      3033.69     558.167        400.254     10.06970   
...               ...          ...         ...            ...          ...   
737448          23.20      2710.94     441.052        386.570      9.62129   
737449          23.20      2692.01     473.436        384.939      9.62063   
737450          23.20      2692.20     500.488        383.496      9.61874   
737451          23.20      1164.12     491.548        384.976      9.61686   
737452          23.20      1164.12     468.019        384.801      9.61497   

        Ore Pulp Density  Flota

In [75]:
# Creating basic regression model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [76]:
#Model coefficients
print("Coefficients: \n", regr.coef_)

Coefficients: 
 [ 1.01574433e-02 -1.60834614e-05  6.54635315e-04  4.55694880e-03
  4.96478455e-02  1.34508638e-01 -4.64233677e-03 -1.55842337e-03
 -1.33750914e-03  2.32477984e-03  1.43404605e-02 -1.30578976e-04
  1.37421996e-03 -2.75924642e-04 -1.68996952e-04 -7.78502573e-05
 -1.41166977e-04 -2.01471685e-04 -1.96899288e-04  1.75980382e-04
 -7.85699132e-01]


In [77]:
#Quality measurement
# mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
#  coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Mean squared error: 0.41
Coefficient of determination: 0.68


El modelo construido es una regresión básica, pero muestra que es posible predecir el porcentaje deconcetrado de  Silice. Con respecto a las preguntas:

1) Sí se puede predecir, sin embargo se debe considerar que esta es solo una versión básica que requiere varias horas de trabajo de preprocesamiento, optimización y análisis para llegar a una versión del modelo que valga la pena para ponerse en producción

2) Si se puede determinar la tiempo con el que se puede predecir, haciendo una autoregresión o moviendo n periodos hacia adeplante la variable objetivo. Dicho análisis requiere varias horas de preprocesamiento que se escapan al alncance de este análisis

3) El coeficiente de correlación entre % Iron Concentrate y % Silica Concentrate es de 0.77. Es una correlación alta y se debe considerar alguna transformación de la variable  